In [9]:
# load the libraries
library(tidycensus)
library(tidyr)
library(stringr)
library(dplyr)


# load your census api key with an environment variable
census_api_key(Sys.getenv("CENSUS_API_KEY"))


# load variable data and transform
variable_data <- load_variables(2020, "acs5", cache = TRUE) %>%
  rename_all(recode,
    name = "variable_key", concept = "dataset",
    label = "variable"
  ) %>%
  mutate(
    dataset = tolower(dataset),
    dataset = gsub(" ", "_", dataset),
    variable = tolower(variable),
    variable = gsub("!!", "_", variable),
    variable = gsub(" ", "_", variable),
    variable = gsub(":", "", variable)
  ) %>%
  select(-geography)


# list of counties
counties <- c(
  "adams", "asotin", "ferry", "garfield", "lincoln",
  "pend oreille", "spokane", "stevens", "whitman"
)


# load census data and transform
getCensusData <- function(table) {
  census_data <- get_acs(
    geography = "tract",
    table = table,
    year = 2020,
    state = "WA",
    survey = "acs5",
    cache_table = FALSE,
    output = "wide"
  ) %>%
    separate(NAME, c("tract", "county", "state"), sep = ",") %>%
    mutate(
      tract = gsub("Census Tract ", "", tract),
      tract = as.double(tract),
      county = tolower(county),
      county = gsub(" county", "", county),
      county = gsub(" ", "", county)
    ) %>%
    select(-state, -GEOID) %>%
    filter(county %in% counties)

  return(census_data)
}


# load urban data and transform
getUrbanCensusData <- function(table) {
  census_data <- get_acs(
    geography = "urban area",
    table = table,
    year = 2020,
    survey = "acs5",
    cache_table = FALSE
  ) %>%
    filter(str_detect(NAME, "Spokane")) %>%
    rename_all(recode, variable = "variable_key") %>%
    merge(variable_data, by = "variable_key") %>%
    select(-GEOID, -NAME, -variable_key)


  return(census_data)
}

naturalization <- getCensusData("B05011")
naturalization_urban <- getUrbanCensusData("B05011")


nativity <- getCensusData("B05012")
nativity_urban <- getUrbanCensusData("B05012")


transportation <- getCensusData("B08101")
transportation_urban <- getUrbanCensusData("B08101")


type_computer <- getCensusData("B28001")
type_computer_urban <- getUrbanCensusData("B28001")


presence_computer <- getCensusData("B28003")
presence_computer_urban <- getUrbanCensusData("B28003")


internet_subscription <- getCensusData("B28011")
internet_subscription_urban <- getUrbanCensusData("B28011")


voting_age <- getCensusData("B29002")
voting_age_urban <- getUrbanCensusData("B29002")


household_income <- getCensusData("S1901")
household_income_urban <- getUrbanCensusData("S1901")



combined_data_urban <- rbind(
  naturalization_urban,
  nativity_urban,
  transportation_urban,
  type_computer_urban,
  presence_computer_urban,
  internet_subscription_urban,
  voting_age_urban,
  household_income_urban
)


dirty_data <- get_acs(
  geography = "tract",
  table = "B28003",
  year = 2020,
  state = "WA",
  survey = "acs5",
  cache_table = FALSE
)

clean_data <- getCensusData("B28003")

To install your API key for use in future sessions, run this function with `install = TRUE`.

Getting data from the 2016-2020 5-year ACS

Getting data from the 2016-2020 5-year ACS

Getting data from the 2016-2020 5-year ACS

Getting data from the 2016-2020 5-year ACS

Getting data from the 2016-2020 5-year ACS

Getting data from the 2016-2020 5-year ACS

Getting data from the 2016-2020 5-year ACS

Getting data from the 2016-2020 5-year ACS

Getting data from the 2016-2020 5-year ACS

Getting data from the 2016-2020 5-year ACS

Getting data from the 2016-2020 5-year ACS

Getting data from the 2016-2020 5-year ACS

Getting data from the 2016-2020 5-year ACS

Getting data from the 2016-2020 5-year ACS

Getting data from the 2016-2020 5-year ACS

Using the ACS Subject Tables

Using the ACS Subject Tables

Using the ACS Subject Tables

Getting data from the 2016-2020 5-year ACS

Using the ACS Subject Tables

Using the ACS Subject Tables

Using the ACS Subject Tables

Getting data from the 

In [13]:
head(naturalization)
head(variable_data)

tract,county,B05011_001E,B05011_001M,B05011_002E,B05011_002M,B05011_003E,B05011_003M,B05011_004E,B05011_004M,⋯,B05011_006E,B05011_006M,B05011_007E,B05011_007M,B05011_008E,B05011_008M,B05011_009E,B05011_009M,B05011_010E,B05011_010M
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
9601,asotin,161,123,142,122,19,21,0,13,⋯,0,13,3,8,0,13,0,13,16,19
9602,asotin,96,82,22,37,74,69,8,18,⋯,0,13,0,13,0,13,47,61,19,31
9603,asotin,96,57,70,44,26,25,21,23,⋯,0,13,0,13,0,13,0,13,5,10
9604,asotin,69,40,36,27,33,32,11,19,⋯,0,13,0,13,6,8,0,13,12,16
9605,asotin,194,84,137,81,57,52,0,13,⋯,0,13,0,13,0,13,0,13,36,33
9606,asotin,89,91,56,86,33,32,0,13,⋯,0,13,8,14,0,13,0,13,13,20


variable_key,variable,dataset
<chr>,<chr>,<chr>
B01001_001,estimate_total,sex_by_age
B01001_002,estimate_total_male,sex_by_age
B01001_003,estimate_total_male_under_5_years,sex_by_age
B01001_004,estimate_total_male_5_to_9_years,sex_by_age
B01001_005,estimate_total_male_10_to_14_years,sex_by_age
B01001_006,estimate_total_male_15_to_17_years,sex_by_age


In [5]:
get_acs(
  geography = "tract",
  variables = 'B05011_001',
  year = 2020,
  state = "WA",
  survey = "acs5",
  cache_table = FALSE
)

Getting data from the 2016-2020 5-year ACS



GEOID,NAME,variable,estimate,moe
<chr>,<chr>,<chr>,<dbl>,<dbl>
53001950100,"Census Tract 9501, Adams County, Washington",B05011_001,122,52
53001950200,"Census Tract 9502, Adams County, Washington",B05011_001,140,131
53001950301,"Census Tract 9503.01, Adams County, Washington",B05011_001,517,222
53001950302,"Census Tract 9503.02, Adams County, Washington",B05011_001,765,254
53001950303,"Census Tract 9503.03, Adams County, Washington",B05011_001,673,268
53001950400,"Census Tract 9504, Adams County, Washington",B05011_001,870,291
53001950500,"Census Tract 9505, Adams County, Washington",B05011_001,1714,363
53003960100,"Census Tract 9601, Asotin County, Washington",B05011_001,161,123
53003960200,"Census Tract 9602, Asotin County, Washington",B05011_001,96,82
